In [1]:
%pip install -q -U google-generativeai
import google.generativeai as genai
import os

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.17.0 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.20.3, but you have protobuf 5.29.0 which is incompatible.

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


/home/maria/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
import pandas as pd

def complete_with_gemini(csv_filename, output_filename):
    """
    Adds a 'gender' column to the input CSV file by querying the Google Cloud API based on the name and location.

    Args:
        csv_filename (str): The input CSV file path.
        output_filename (str): The output CSV file path with the new 'gender' column.

    Returns:
        pd.DataFrame: The updated DataFrame with the 'gender' column.
    """

    os.environ["GEMINI_API_KEY"] = ''
    genai.configure(api_key=os.environ["GEMINI_API_KEY"])

    # Read the CSV file
    df = pd.read_csv(csv_filename)

    model = genai.GenerativeModel("gemini-1.5-flash")

    # Add a 'gender' column to the DataFrame
    genders = []
    for _, row in df.iterrows():
        name = row[' user']
        location = row.get(' location', '')  # Default to an empty string if 'location' is missing

        # Construct the prompt
        prompt = f"""
        Given the name '{name}' and, if not none, the location '{location}',
        tell me the gender of the username.
        - The input is a name and maybe a location too.
        - The output must be 'Male', 'Female', or
        'Unknown'. Without any explanations or reasons. Just the predicted gender.
        - Examples:
          - Christopher Pirillo: Male
          - Vera AxelRod: Female
          - Vish Ganapathy: Unknown
        """

        try:

            generation_config = {
              "temperature": 1,
              "top_p": 0.95,
              "top_k": 64,
              "max_output_tokens": 8192,
              "response_mime_type": "text/plain",
              }

            response = model.generate_content(prompt, generation_config=generation_config)
            print(response.text)

            # Parse the result
            result = response.text
            genders.append(result)

        except Exception as e:
            print(f"Error processing {name}: {e}")
            genders.append("Error")  # Append "Error" in case of failure

    # Add the genders to the DataFrame
    df['gender-google'] = genders

    # Save the updated DataFrame to a new CSV file
    df.to_csv(output_filename, index=False)

    return df

In [3]:
import pandas as pd
complete_with_gemini("pilot.csv", 'pilot_gemini.csv')

Unknown

Male

Male

Male

Male

Male

Male

Unknown

Male

Female

Male

Male

Unknown

Male

Male

Unknown

Male

Unknown

Female

Unknown

Unknown

Male

Unknown

Male

Male

Male

Male

Male

Unknown

Male

Male

Male

Male

Unknown

Male

Male

Male

Male

Male

Male

Unknown

Male

Male

Male

Unknown

Male

Male

Male

Male

Unknown

Male

Unknown

Male

Female

Male

Male

Unknown

Unknown

Unknown

Unknown

Male

Male

Unknown

Male

Unknown

Unknown

Male

Male

Male

Male

Unknown

Unknown

Unknown

Male

Male

Male

Male

Unknown

Male

Male

Male

Unknown

Male

Female

Unknown

Female

Unknown

Unknown

Male

Unknown

Unknown

Male

Unknown

Unknown

Unknown

Male

Unknown

Male

Male

Unknown

Unknown

Male

Unknown

Unknown

Male

Male

Male

Unknown

Unknown

Unknown

Unknown

Unknown

Unknown

Male

Male

Unknown

Unknown

Unknown

Male

Female

Male

Male

Unknown

Male

Male

Male

Male

Unknown

Male

Unknown

Unknown

Male

Unknown

Male

Male

Male

Male

Unknown

,username,user,location,gender-google
0,zhiqings,Zhiqing Sun,USA,Unknown\n
1,jysohn23,Daniel JinYoung Sohn,USA,Male\n
2,LudovicPeran,LUDOVIC PERAN,NaN,Male\n
3,jwexler,James Wexler,NaN,Male\n
4,iftenney,Ian Tenney,USA,Male\n
...,...,...,...,...
354,Mukul1,Mukul Chaudhary,NaN,Male\n
355,jorsj,Jorge Sierra,NaN,Male\n
356,awebson,Albert Webson,NaN,Male\n
357,lauraio,Laura Ionita,NaN,Female\n


In [25]:
import pandas as pd
df = pd.read_csv('pilot_complete.csv')

df = df.drop(columns=[' Gender-openai', ' Gender-google', 'gender-Anthropic'])

In [12]:
df

,username,user,ubication,gender
0,zhiqings,Zhiqing Sun,USA,male
1,jysohn23,Daniel JinYoung Sohn,USA,male
2,LudovicPeran,LUDOVIC PERAN,,male
3,jwexler,James Wexler,,male
4,iftenney,Ian Tenney,USA,male
...,...,...,...,...
354,Mukul1,Mukul Chaudhary,,male
355,jorsj,Jorge Sierra,NaN,male
356,awebson,Albert Webson,NaN,male
357,lauraio,Laura Ionita,NaN,female


In [26]:
df2 = pd.read_csv('pilot_gemini.csv')
df2['gender-google'] = df2['gender-google'].str.replace('\n', '').str.lower()
df2

,username,user,location,gender-google
0,zhiqings,Zhiqing Sun,USA,unknown
1,jysohn23,Daniel JinYoung Sohn,USA,male
2,LudovicPeran,LUDOVIC PERAN,NaN,male
3,jwexler,James Wexler,NaN,male
4,iftenney,Ian Tenney,USA,male
...,...,...,...,...
354,Mukul1,Mukul Chaudhary,NaN,male
355,jorsj,Jorge Sierra,NaN,male
356,awebson,Albert Webson,NaN,male
357,lauraio,Laura Ionita,NaN,female


In [27]:
df = df[' gender'].str.strip()
df2 = df2['gender-google']
comparison_df = pd.concat([df, df2], axis=1)
comparison_df['Match'] = comparison_df[' gender'] == comparison_df['gender-google']
comparison_df

,gender,gender-google,Match
0,male,unknown,False
1,male,male,True
2,male,male,True
3,male,male,True
4,male,male,True
...,...,...,...
354,male,male,True
355,male,male,True
356,male,male,True
357,female,female,True


In [30]:
false_count = (comparison_df['Match'] == False).sum()
false_count

96

In [31]:
print(96/359)

0.26740947075208915
